# Day 15: Rambunctious Recitation

You catch the airport shuttle and try to book a new flight to your vacation island. Due to the storm, all direct flights have been cancelled, but a route is available to get around the storm. You take it.

While you wait for your flight, you decide to check in with the Elves back at the North Pole. They're playing a memory game and are ever so excited to explain the rules!

In this game, the players take turns saying numbers. They begin by taking turns reading from a list of starting numbers (your puzzle input). Then, each turn consists of considering the most recently spoken number:

    If that was the first time the number has been spoken, the current player says 0.
    Otherwise, the number had been spoken before; the current player announces how many turns apart the number is from when it was previously spoken.

So, after the starting numbers, each turn results in that player speaking aloud either 0 (if the last number is new) or an age (if the last number is a repeat).

For example, suppose the starting numbers are 0,3,6:

    Turn 1: The 1st number spoken is a starting number, 0.
    Turn 2: The 2nd number spoken is a starting number, 3.
    Turn 3: The 3rd number spoken is a starting number, 6.
    Turn 4: Now, consider the last number spoken, 6. Since that was the first time the number had been spoken, the 4th number spoken is 0.
    Turn 5: Next, again consider the last number spoken, 0. Since it had been spoken before, the next number to speak is the difference between the turn number when it was last spoken (the previous turn, 4) and the turn number of the time it was most recently spoken before then (turn 1). Thus, the 5th number spoken is 4 - 1, 3.
    Turn 6: The last number spoken, 3 had also been spoken before, most recently on turns 5 and 2. So, the 6th number spoken is 5 - 2, 3.
    Turn 7: Since 3 was just spoken twice in a row, and the last two turns are 1 turn apart, the 7th number spoken is 1.
    Turn 8: Since 1 is new, the 8th number spoken is 0.
    Turn 9: 0 was last spoken on turns 8 and 4, so the 9th number spoken is the difference between them, 4.
    Turn 10: 4 is new, so the 10th number spoken is 0.

(The game ends when the Elves get sick of playing or dinner is ready, whichever comes first.)

Their question for you is: what will be the 2020th number spoken? In the example above, the 2020th number spoken will be 436.

Here are a few more examples:

    Given the starting numbers 1,3,2, the 2020th number spoken is 1.
    Given the starting numbers 2,1,3, the 2020th number spoken is 10.
    Given the starting numbers 1,2,3, the 2020th number spoken is 27.
    Given the starting numbers 2,3,1, the 2020th number spoken is 78.
    Given the starting numbers 3,2,1, the 2020th number spoken is 438.
    Given the starting numbers 3,1,2, the 2020th number spoken is 1836.

Given your starting numbers, what will be the 2020th number spoken?

Your puzzle input is 0,14,1,3,7,9.

In [1]:
# Python imports
from pathlib import Path
from typing import List

from tqdm.notebook import tqdm

As 2020 is a small number of rounds, we can keep the list of spoken numbers in memory, and check whether the last number in the list has been spoken before, directly.

The only unusual thing here is

```python
idx = len(numbers) - numbers[-2::-1].index(num) - 1
```

where we find the last index of the number `num` in the list `numbers` by inverting the list and using the result's `.index()` method (Python doesn't have a way to find this directly).

In [2]:
def age_game(invals: List[int], lastnum: int=2020) -> int:
    """Return the last number spoken in the game
    
    :param invals:  starting sequence
    :param lastnum:  number of rounds in the game
    """
    numbers = invals[:]  # holds all rounds of the game
            
    for rnd in range(len(invals), lastnum):  # play game
        num = numbers[-1]  # the most recently spoken number
        if numbers.count(num) == 1:  # first time spoken
            numbers.append(0)
        else:  # has been spoken before
            idx = len(numbers) - numbers[-2::-1].index(num) - 1
            numbers.append(rnd - idx)
            
    return numbers

In [3]:
numbers = age_game([0,3,6])
numbers[-1]

436

In [4]:
numbers = age_game([3,1,2])
numbers[-1]

1836

In [5]:
numbers = age_game([0,14,1,3,7,9])
numbers[-1]

763

# Part Two

Impressed, the Elves issue you a challenge: determine the 30000000th number spoken. For example, given the same starting numbers as above:

    Given 0,3,6, the 30000000th number spoken is 175594.
    Given 1,3,2, the 30000000th number spoken is 2578.
    Given 2,1,3, the 30000000th number spoken is 3544142.
    Given 1,2,3, the 30000000th number spoken is 261214.
    Given 2,3,1, the 30000000th number spoken is 6895259.
    Given 3,2,1, the 30000000th number spoken is 18.
    Given 3,1,2, the 30000000th number spoken is 362.

Given your starting numbers, what will be the 30000000th number spoken?

The number of rounds in the game is too large to search the list, now. So we record all numbers that have been seen before as keys in a dictionary `ages`, where the associated value is the last round the number was seen. This makes the search for the last-seen round much more efficient.

In [6]:
def age_game2(invals: List[int], lastnum: int=2020) -> int:
    """Return the (lastnum)th number spoken in the game
    
    :param invals:  the starting number list
    :param lastnum:  the number of rounds in the game
    """
    ages = {}  # last-seen round for each number
    for idx, val in enumerate(invals[:-1]):  # populate ages
        ages[val] = idx

    numbers = invals[:]  # holds rounds in the game
        
    for rnd in tqdm(range(len(ages), lastnum-1)):
        if numbers[-1] in ages:  # number has been seen
            newage = rnd - ages[numbers[-1]]
            ages[numbers[-1]] = rnd
            numbers.append(newage)
        else:  # new number
            ages[numbers[-1]] = rnd
            numbers.append(0)
    
    return numbers[-1]  # last number in game

In [7]:
numbers = age_game([0,3,6])
numbers[-1]

436

In [8]:
num = age_game2([0,3,6])
num

436

In [9]:
num = age_game2([0,14,1,3,7,9])
num

763

In [10]:
num = age_game2([0,3,6], 30000000)
num

175594

In [11]:
num = age_game2([0,14,1,3,7,9], 30000000)
num

1876406